In [1]:
import pandas as pd
import sklearn
# from sklearn import cross_validation
# from sklearn.cross_validation import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
dataset = pd.read_excel(r'./200416_label0_dataset_sentence.xlsx')

In [2]:
tmp = []
for i in dataset['sender']:
    if i == 'customer':
        tmp.append(0)
    else:
        tmp.append(1)
dataset['sender_num'] = tmp
dataset.head()

,id,round,sender,texts,max_label,round_label,labels,sender_num
0,4227729258237820,1,customer,内涵 段子 联通 皮 点赞 中国联通 中国联通 客服 掌上 营业厅 内涵 段子 话题 封 郑...,3,2,1,0
1,4227729258237820,2,helpdesk,u,6,4,0,1
2,4227729258237820,3,customer,夸夸,3,0,1,0
3,4227729258237820,4,helpdesk,*,6,4,0,1
4,4121001149457180,1,customer,距离 反映 问题 已经 一个 星期 花粉 助手 D 荣耀 honor 荣耀 手机 华为 终端...,2,2,1,0


In [3]:
texts = list(dataset["texts"])
# labels = list(dataset["labels"])
max_label = list(dataset["max_label"])
Round = list(dataset["round"])
sender_num = list(dataset["sender_num"])

In [6]:
x_train=texts[1755:]
x_test=texts[0:1755]
y_train=max_label[1755:]
y_test=max_label[0:1755]

In [4]:
import os
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.decomposition import TruncatedSVD

In [5]:
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# from xgboost import XGBClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()

In [7]:
def baseline_model(x_train, x_test, y_train, y_test):
    
    x_train = tfidf_vectorizer.fit_transform(x_train)
    x_test = tfidf_vectorizer.transform(x_test)
    
    Models = ['Bernoulli NB','Multinomial NB','Svm (linear)','Logistic Regression',
              'Random Forest','kNN','Decision Tree']
    function = [BernoulliNB(),MultinomialNB(),svm.SVC(kernel="linear"),LogisticRegression(),
              RandomForestClassifier(),KNeighborsClassifier(),DecisionTreeClassifier()]
    perform_f1 = []
    perform_acc = []
    
    
    for i in tqdm(range(len(function))):
        model = function[i]
#         performance = cross_val_score(model, tfidf_vectorizer.fit_transform(texts), labels, cv=10, scoring'accuracy')
        
        func = str(function[i])
        print("==== ", func[0:func.index('(')], " ====")

        model.fit(x_train, y_train)
        model.score(x_test, y_test)
        e = y_test
        p = model.predict(x_test)
        print(metrics.classification_report(e,p))
        perform_f1.append(metrics.f1_score(e,p,average='macro'))
        perform_acc.append(metrics.accuracy_score(e,p))
        
    result_f1_table = pd.DataFrame({"Models":Models,"Result f1 scores":perform_f1})
    result_acc_table = pd.DataFrame({"Models":Models,"Result acc scores":perform_acc})
    return result_f1_table, result_acc_table

In [8]:
score = baseline_model(x_train, x_test, y_train, y_test)

====  BernoulliNB  ====
              precision    recall  f1-score   support

           0       0.63      0.88      0.74       477
           1       0.00      0.00      0.00        23
           2       0.88      0.88      0.88       374
           3       1.00      0.01      0.02       101
           4       0.84      0.90      0.87       662
           5       0.00      0.00      0.00        33
           6       1.00      0.01      0.02        85

    accuracy                           0.77      1755
   macro avg       0.62      0.38      0.36      1755
weighted avg       0.78      0.77      0.72      1755

====  MultinomialNB  ====
              precision    recall  f1-score   support

           0       0.74      0.38      0.50       477
           1       0.00      0.00      0.00        23
           2       0.71      0.94      0.81       374
           3       0.00      0.00      0.00       101
           4       0.64      0.98      0.78       662
           5       0.00     

/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.69      0.87      0.77       477
           1       0.67      0.17      0.28        23
           2       0.87      0.96      0.91       374
           3       0.25      0.03      0.05       101
           4       0.87      0.91      0.89       662
           5       0.00      0.00      0.00        33
           6       0.74      0.24      0.36        85

    accuracy                           0.80      1755
   macro avg       0.58      0.45      0.47      1755
weighted avg       0.76      0.80      0.76      1755

====  LogisticRegression  ====


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


              precision    recall  f1-score   support

           0       0.68      0.88      0.77       477
           1       0.75      0.13      0.22        23
           2       0.87      0.95      0.91       374
           3       0.22      0.02      0.04       101
           4       0.87      0.91      0.89       662
           5       0.33      0.03      0.06        33
           6       0.86      0.22      0.36        85

    accuracy                           0.80      1755
   macro avg       0.66      0.45      0.46      1755
weighted avg       0.77      0.80      0.76      1755

====  RandomForestClassifier  ====


/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.73      0.74      0.73       477
           1       0.40      0.17      0.24        23
           2       0.89      0.98      0.93       374
           3       0.36      0.05      0.09       101
           4       0.84      0.91      0.88       662
           5       0.00      0.00      0.00        33
           6       0.33      0.45      0.38        85

    accuracy                           0.78      1755
   macro avg       0.51      0.47      0.46      1755
weighted avg       0.75      0.78      0.76      1755

====  KNeighborsClassifier  ====
              precision    recall  f1-score   support

           0       0.37      0.51      0.43       477
           1       0.21      0.13      0.16        23
           2       0.83      0.05      0.10       374
           3       0.05      0.15      0.07       101
           4       0.84      0.41      0.55       662
           5       0.33      0.03      0.06  

In [18]:
answer1=p.tolist()

In [19]:
answer1[:10]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [20]:
import pandas as pd
dict1 = {"label": answer1,  
       }
select_df = pd.DataFrame(dict1)

In [21]:
select_df.to_csv('answer1.csv')